In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from pgmpy.models import BayesianNetwork
from pgmpy.estimators import MaximumLikelihoodEstimator, K2Score, HillClimbSearch
from pgmpy.readwrite import BIFWriter
import numpy as np
import dask.dataframe as dd

In [ ]:
#DIABETE TIPO 1

In [ ]:
dfTDM1 = pd.read_csv ('T1DM.csv')
dfTDM1 = dfTDM1.drop('Unnamed: 0', axis = 1)

In [ ]:
# Dividere il dataset in train e test
train_data, test_data = train_test_split(dfTDM1, test_size=0.2, random_state=42)

In [ ]:
# Definire la struttura della rete bayesiana
# Utilizzare Hill-Climbing per apprendere la struttura
dfTDM1 = dfTDM1.astype({col: 'float32' for col in dfTDM1.select_dtypes(include='float64').columns})
hc = HillClimbSearch(dfTDM1)
best_model_structure = hc.estimate(max_iter = 100000, scoring_method=K2Score(dfTDM1), max_indegree = 2)
print("Archi della struttura appresa:", best_model_structure.edges())

# Verifica che best_model.edges() non sia None
if best_model_structure.edges() is None:
    raise ValueError("La struttura della rete appresa è vuota.")
# Stampa la struttura appresa
#print("Archi della struttura appresa:", best_model.edges())

# Creare il modello con la struttura appresa
try:
    model = BayesianNetwork(best_model_structure.edges())
except Exception as e:
    print(f'Errore nella creazione del modello di rete bayesiana: {e}')
    raise
print(model)
print("modello creato")

# Verifica che il modello sia stato creato correttamente
if model is None:
    raise ValueError("Errore nella creazione del modello di rete bayesiana.")

# Addestrare la rete bayesiana con i dati di training
try:
    model.fit(train_data, estimator=MaximumLikelihoodEstimator)
except Exception as e:
    print(f'Errore durante l\'addestramento del modello: {e}')
    raise

print("Modello addestrato e salvato con successo.")

In [ ]:
for cpd in model.get_cpds():
    cpd.to_csv(filename=str(cpd.get_evidence())+".csv")
    

In [ ]:
# Reset indici
df_test_data = test_data.reset_index(drop=True)

In [ ]:
df_test_data.shape
train_data.shape

In [ ]:
from pgmpy.inference import VariableElimination
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Supponendo che 'model' e 'df_test_data' siano già definiti

# Creare un oggetto per l'inferenza
inference = VariableElimination(model)

# Separare le features e la variabile target
X_test = df_test_data.drop(columns=['DIABTYPE1']).copy()
y_true = df_test_data['DIABTYPE1'].values
y_pred_total = []

# Funzione per salvare le previsioni su file
def salva_previsioni(y_pred, blocco):
    print(f'Saving: blocco {blocco}')
    with open(f'previsioni_blocco_{blocco}.txt', 'w') as file:
        for pred in y_pred:
            file.write(f"{pred}\n")

# Iterare il dataframe a blocchi di 100 righe
blocco = 0
for start in range(0, len(X_test), 100):
    end = start + 100
    df_blocco = X_test.iloc[start:end]
    print(f'Prediction per: {start}:{end}, blocco {blocco}')
    
    y_pred = []
    for index, row in df_blocco.iterrows():
        print(f'Prediction for row index: {index}, blocco {blocco}')
        evidence = row.to_dict()
        
        # Validazione delle evidenze
        try:
            for node, value in evidence.items():
                cpd = model.get_cpds(node)
                valid_states = cpd.state_names[node]
                if value not in valid_states:
                    raise ValueError(f"Lo stato '{value}' non è valido per il nodo '{node}'")
        except Exception as e:
            print(f'Validazione evidenza fallita per index {index}: {e}')
            y_pred.append(None)
            continue
        
        # Inferenza
        try:
            prediction = inference.map_query(variables=['DIABTYPE1'], evidence=evidence)
            print(f'Prediction per index {index}: {prediction}')
            
            if isinstance(prediction, dict) and 'DIABTYPE1' in prediction:
                y_pred.append(prediction['DIABTYPE1'])
            else:
                print(f'Predizione non valida per index {index}: {prediction}')
                y_pred.append(None)
                
        except Exception as e:
            print(f'Errore al processing dell\'indice {index}: {e}')
            y_pred.append(None)
    
    # Salvare le previsioni su file dopo ogni blocco di 100 righe
    salva_previsioni(y_pred, blocco)
    y_pred_total.extend(y_pred)
    blocco += 1

# Filtrare le previsioni valide
y_pred_final = []
y_true_final = []
for pred, true in zip(y_pred_total, y_true):
    if pred is not None:
        y_pred_final.append(pred)
        y_true_final.append(true)



In [ ]:
y_pred_final

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Rimuovere None dai predetti e dai veri valori corrispondenti
y_true_filtered = [yt for yt, yp in zip(y_true, y_pred) if yp is not None]
y_pred_filtered = [yp for yp in y_pred if yp is not None]

# Calcolare le metriche di valutazione
accuracy = accuracy_score(y_true_filtered, y_pred_filtered)
precision = precision_score(y_true_filtered, y_pred_filtered, average='macro')  # O 'micro' o 'weighted' a seconda del tuo caso
recall = recall_score(y_true_filtered, y_pred_filtered, average='macro')  # O 'micro' o 'weighted'
f1 = f1_score(y_true_filtered, y_pred_filtered, average='macro')  # O 'micro' o 'weighted'

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')


In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
def plot_bayesian_network(model):
    G = nx.DiGraph()

    # Aggiungi nodi e archi
    G.add_nodes_from(model.nodes())
    G.add_edges_from(model.edges())

    # Disegna la rete
    pos = nx.spring_layout(G)
    nx.draw(G, pos, with_labels=True, node_size=500, node_color="skyblue", font_size=4, font_color="black", font_weight="bold", arrowsize=5)
    plt.savefig('rete_tipo1.jpeg', format="jpeg")
    plt.show()

plot_bayesian_network(model)

In [ ]:
#DIABETE TIPO 2

In [ ]:
dfTDM2 = pd.read_csv ('T2DM.csv')
dfTDM2 = dfTDM2.drop('Unnamed: 0', axis = 1)

In [ ]:
# Dividere il dataset in train e test
train_data, test_data = train_test_split(dfTDM2, test_size=0.2, random_state=42)

In [ ]:
train_data.shape

In [ ]:
test_data.shape

In [ ]:
# Definire la struttura della rete bayesiana
# Utilizzare Hill-Climbing per apprendere la struttura
dfTDM2 = dfTDM2.astype({col: 'float32' for col in dfTDM2.select_dtypes(include='float64').columns})
hc = HillClimbSearch(dfTDM2)
best_model_structure = hc.estimate(max_iter = 100000, scoring_method=K2Score(dfTDM2), max_indegree = 2)
print("Archi della struttura appresa:", best_model_structure.edges())

# Verifica che best_model.edges() non sia None
if best_model_structure.edges() is None:
    raise ValueError("La struttura della rete appresa è vuota.")
# Stampa la struttura appresa
#print("Archi della struttura appresa:", best_model.edges())

# Creare il modello con la struttura appresa
try:
    model = BayesianNetwork(best_model_structure.edges())
except Exception as e:
    print(f'Errore nella creazione del modello di rete bayesiana: {e}')
    raise
print(model)
print("modello creato")

# Verifica che il modello sia stato creato correttamente
if model is None:
    raise ValueError("Errore nella creazione del modello di rete bayesiana.")

# Addestrare la rete bayesiana con i dati di training
try:
    model.fit(train_data, estimator=MaximumLikelihoodEstimator)
except Exception as e:
    print(f'Errore durante l\'addestramento del modello: {e}')
    raise

print("Modello addestrato e salvato con successo.")

In [ ]:
# Reset indici
df_test_data = test_data.reset_index(drop=True)

In [ ]:
from pgmpy.inference import VariableElimination
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Supponendo che 'model' e 'df_test_data' siano già definiti

# Creare un oggetto per l'inferenza
inference = VariableElimination(model)

# Separare le features e la variabile target
X_test = df_test_data.drop(columns=['DIABTYPE2']).copy()
y_true = df_test_data['DIABTYPE2'].values
y_pred_total = []

# Funzione per salvare le previsioni su file
def salva_previsioni(y_pred, blocco):
    print(f'Saving: blocco {blocco}')
    with open(f'previsioni_blocco_{blocco}.txt', 'w') as file:
        for pred in y_pred:
            file.write(f"{pred}\n")

# Iterare il dataframe a blocchi di 100 righe
blocco = 0
for start in range(0, len(X_test), 100):
    end = start + 100
    df_blocco = X_test.iloc[start:end]
    print(f'Prediction per: {start}:{end}, blocco {blocco}')
    
    y_pred = []
    for index, row in df_blocco.iterrows():
        print(f'Prediction for row index: {index}, blocco {blocco}')
        evidence = row.to_dict()
        
        # Validazione delle evidenze
        try:
            for node, value in evidence.items():
                cpd = model.get_cpds(node)
                valid_states = cpd.state_names[node]
                if value not in valid_states:
                    raise ValueError(f"Lo stato '{value}' non è valido per il nodo '{node}'")
        except Exception as e:
            print(f'Validazione evidenza fallita per index {index}: {e}')
            y_pred.append(None)
            continue
        
        # Inferenza
        try:
            prediction = inference.map_query(variables=['DIABTYPE2'], evidence=evidence)
            print(f'Prediction per index {index}: {prediction}')
            
            if isinstance(prediction, dict) and 'DIABTYPE2' in prediction:
                y_pred.append(prediction['DIABTYPE2'])
            else:
                print(f'Predizione non valida per index {index}: {prediction}')
                y_pred.append(None)
                
        except Exception as e:
            print(f'Errore al processing dell\'indice {index}: {e}')
            y_pred.append(None)
    
    # Salvare le previsioni su file dopo ogni blocco di 100 righe
    salva_previsioni(y_pred, blocco)
    y_pred_total.extend(y_pred)
    blocco += 1

# Filtrare le previsioni valide
y_pred_final = []
y_true_final = []
for pred, true in zip(y_pred_total, y_true):
    if pred is not None:
        y_pred_final.append(pred)
        y_true_final.append(true)



In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Rimuovere None dai predetti e dai veri valori corrispondenti
y_true_filtered = [yt for yt, yp in zip(y_true, y_pred) if yp is not None]
y_pred_filtered = [yp for yp in y_pred if yp is not None]

# Calcolare le metriche di valutazione
accuracy = accuracy_score(y_true_filtered, y_pred_filtered)
precision = precision_score(y_true_filtered, y_pred_filtered, average='macro')  # O 'micro' o 'weighted' a seconda del tuo caso
recall = recall_score(y_true_filtered, y_pred_filtered, average='macro')  # O 'micro' o 'weighted'
f1 = f1_score(y_true_filtered, y_pred_filtered, average='macro')  # O 'micro' o 'weighted'

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')


In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
def plot_bayesian_network(model):
    G = nx.DiGraph()

    # Aggiungi nodi e archi
    G.add_nodes_from(model.nodes())
    G.add_edges_from(model.edges())

    # Disegna la rete
    pos = nx.spring_layout(G)
    nx.draw(G, pos, with_labels=True, node_size=500, node_color="skyblue", font_size=4, font_color="black", font_weight="bold", arrowsize=5)
    plt.savefig('rete_tipo2.jpeg', format="jpeg")
    plt.show()

plot_bayesian_network(model)